In [1]:
!pip install flask transformers torch requests pyngrok


In [9]:
GROQ_API_KEY = "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"


In [10]:
from flask import Flask, request, render_template_string
import os
import requests
from transformers import pipeline
from pyngrok import ngrok

app = Flask(__name__)
OUTPUT_FOLDER = "summaries"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Load local summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Transcription using Groq
def transcribe_audio(file_path):
    url = "https://api.groq.com/openai/v1/audio/transcriptions"
    headers = {"Authorization": f"Bearer {GROQ_API_KEY}"}
    files = {"file": open(file_path, "rb")}
    data = {"model": "whisper-large-v3-turbo", "response_format": "text"}
    response = requests.post(url, headers=headers, files=files, data=data)
    files['file'].close()
    if response.status_code == 200:
        return response.text.strip()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Summarization
def summarize_transcript(transcript):
    transcript_chunk = transcript[:4000] if len(transcript) > 4000 else transcript
    summary = summarizer(transcript_chunk, max_length=300, min_length=80, do_sample=False)[0]['summary_text']
    return summary

# Frontend + API
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Meeting Summarizer</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { padding: 40px; background-color: #f7f9fc; }
        h1 { margin-bottom: 30px; }
        .card { margin-top: 20px; }
        .spinner { display: none; }
    </style>
</head>
<body>
<div class="container">
    <h1 class="text-center">Meeting Summarizer</h1>
    <form method="POST" enctype="multipart/form-data" class="text-center">
        <input type="file" name="file" class="form-control mb-3" required>
        <button type="submit" class="btn btn-primary">Upload & Summarize</button>
    </form>

    {% if transcript %}
    <div class="card">
        <div class="card-header bg-info text-white">Transcript</div>
        <div class="card-body"><pre>{{ transcript }}</pre></div>
    </div>

    <div class="card">
        <div class="card-header bg-success text-white">Summary & Action Items</div>
        <div class="card-body"><pre>{{ summary }}</pre></div>
    </div>
    {% endif %}
</div>
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    transcript, summary = None, None
    if request.method == "POST":
        uploaded_file = request.files["file"]
        file_path = os.path.join(OUTPUT_FOLDER, uploaded_file.filename)
        uploaded_file.save(file_path)

        transcript = transcribe_audio(file_path)
        if "Error" not in transcript:
            summary = summarize_transcript(transcript)
    return render_template_string(HTML_TEMPLATE, transcript=transcript, summary=summary)


Device set to use cpu


In [11]:
!ngrok authtoken 345mv7zeN3GBKWBzHbyA4vZz7Fn_6N3t9sAQMB4QHr1Yv36bA


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [12]:
from threading import Thread

def run_app():
    app.run(host="0.0.0.0", port=5000, threaded=True, use_reloader=False)

# Start Flask in a background thread
thread = Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [13]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print(f"Access your app at: {public_url}")


Access your app at: NgrokTunnel: "https://unfelicitated-brandon-semitraditionally.ngrok-free.dev" -> "http://localhost:5000"
